In [ ]:
import pandas as pd
import re
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, set_seed
import transformers
import accelerate

In [ ]:
data = pd.read_csv("data/goodreads_books.csv", converters={'COLUMN_NAME': pd.eval})

In [ ]:
data = data.filter(["title", "rating_count", "average_rating", "genre_and_votes"], axis=1)

In [ ]:
def genre_break(genre):
    for index, item in enumerate(genre):
        new_item = re.sub('\s[0-9]+', '', item)
        genre[index] = new_item
    
    return genre

In [ ]:
data["genre"] = data["genre_and_votes"].str.split(", ")
data.drop(axis=1, columns=["genre_and_votes"], inplace=True)

In [ ]:
data.dropna(inplace=True, axis=0)

In [ ]:
data.info()

In [ ]:
data["genre"].apply(lambda item: genre_break(item))

In [ ]:
data

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

In [ ]:
set_seed(32)
text = "Generate a book title for a new sci-fi series."
encoded_input = tokenizer.encode(text, return_tensors='pt')
outputs = model.generate(encoded_input, max_length = 40, no_repeat_ngram_size = 2, early_stopping = True, num_return_sequences = 5, num_beams=5,)

In [ ]:
for output in outputs:
  print(tokenizer.decode(output, skip_special_tokens=True))